In [ ]:
# !pip install -q pypdf
# !pip install -q python-dotenv
# !pip install -q llama-index
# !pip install -q gradio
# !pip install einops
# !pip install accelerate
# !pip install faiss-cpu
# !pip install faiss-gpu

# !pip install langchain
# !pip install sentence_transformers
# !pip uninstall -y transformers
# !pip install git+https://github.com/huggingface/transformers

In [ ]:
from llama_index.llms import HuggingFaceLLM
import torch
llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.5, "do_sample": True},
    tokenizer_name="microsoft/phi-2",
    model_name="microsoft/phi-2",
    device_map="cuda",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.bfloat16}
)

In [ ]:
# from llama_index import Document
#from llama_index.embeddings import OpenAIEmbedding
# from llama_index.text_splitter import SentenceSplitter
# from llama_index.extractors import TitleExtractor
# from llama_index.ingestion import IngestionPipeline, IngestionCache
# from llama_index import VectorStoreIndex

# import os
# # os.environ["OPENAI_API_KEY"] = "random"

from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
# from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding
from llama_index import set_global_service_context
# from llama_index import (
#     ServiceContext
# )




# index = VectorStoreIndex.from_documents(
#     [Document.example()]
# )

sudo -i -u postgres

# Loading data in Faiss Vector store
- Creating a Faiss Index
- Load documents, build the VectorStoreIndex
- Download and save data to Faiss store

In [ ]:
from llama_index import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.vector_stores.faiss import FaissVectorStore
# from IPython.display import Markdown, display
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM
import torch
import faiss

# dimensions of text-ada-embedding-002
d = 768
faiss_index = faiss.IndexFlatL2(d)



In [ ]:
# from llama_index import Document
# from llama_index.embeddings import OpenAIEmbedding
# from llama_index.text_splitter import SentenceSplitter
# from llama_index.extractors import TitleExtractor
# from llama_index.ingestion import IngestionPipeline, IngestionCache

# # create the pipeline with transformations
# pipeline = IngestionPipeline(
#     transformations=[
#         SentenceSplitter(chunk_size=25, chunk_overlap=0),
#         TitleExtractor(),
#         OpenAIEmbedding(),
#     ]
# )

# # run the pipeline
# nodes = pipeline.run(documents=[Document.example()])

In [ ]:
# load documents
documents = SimpleDirectoryReader("./data/").load_data()
vector_store = FaissVectorStore(faiss_index=faiss_index)

In [ ]:
embed_model = LangchainEmbedding(
  HuggingFaceBgeEmbeddings(model_name="BAAI/bge-base-en")
)

In [ ]:
service_context = ServiceContext.from_defaults(
    embed_model=embed_model,
    llm=llm,
)

In [ ]:
set_global_service_context(service_context)

In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)

In [ ]:
# save index to disk
index.storage_context.persist()

# Loading data from Faiss store

In [33]:
from llama_index.prompts.prompts import SimpleInputPrompt
from llama_index.llms import HuggingFaceLLM
import torch

system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")



llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=256,
    # generate_kwargs={"temperature": 0.5, "do_sample": False},
    # system_prompt=system_prompt,
    # query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.2",
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    device_map="cuda",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.bfloat16}
)

from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding
from llama_index import set_global_service_context

# loads BAAI/bge-small-en
# embed_model = HuggingFaceEmbedding()

# loads BAAI/bge-small-en-v1.5

embed_model = LangchainEmbedding(
  HuggingFaceBgeEmbeddings(model_name="BAAI/bge-base-en")
)

service_context = ServiceContext.from_defaults(
    embed_model=embed_model,
    llm=llm,
    chunk_size=1024,
)

set_global_service_context(service_context)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [34]:
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index import StorageContext
from llama_index import ServiceContext
from llama_index import set_global_service_context
from llama_index import load_index_from_storage
from llama_index.embeddings import LangchainEmbedding
from langchain.embeddings.huggingface import HuggingFaceBgeEmbeddings

# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./storage")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./storage"
)
index = load_index_from_storage(storage_context=storage_context,service_context=service_context)

## General QA prompt with phi-2

In [35]:
from llama_index import SimpleDirectoryReader, VectorStoreIndex

In [36]:
# documents = SimpleDirectoryReader("./data/").load_data()

In [37]:
# index = VectorStoreIndex.from_documents(documents, service_context=service_context)

query_engine = index.as_query_engine()
chat_engine = index.as_chat_engine()

def predict(input, history):
  response = query_engine.query(input)
  return str(response)


import gradio as gr

gr.ChatInterface(predict).launch(share=True)

In [38]:
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.retrievers import VectorIndexRetriever
from llama_index.response_synthesizers import get_response_synthesizer
retriever = VectorIndexRetriever(
      index=index,
      similarity_top_k=2,
)
response_synthesizer = get_response_synthesizer(
      response_mode="tree_summarize",
)
query_engine = RetrieverQueryEngine(
      retriever=retriever,
      response_synthesizer=response_synthesizer,
)

In [39]:
# chat_engine = index.as_chat_engine(chat_mode='react', verbose=True)
# response = chat_engine.chat('what is the name of the movie?')

In [40]:
response = query_engine.query('what is the name of the movie? Also provide the name of Director and the year of release along with the writer of the movie. Also provide the genre and actors of the movie. Do not include any other additional information')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [41]:
chat_engine.chat('What is this document about?')

TemplateError: Conversation roles must alternate user/assistant/user/assistant/...

In [42]:
chat_engine = index.as_chat_engine(chat_mode='react', verbose=True)
response = chat_engine.chat('user:What is this document about?')

TemplateError: Conversation roles must alternate user/assistant/user/assistant/...

In [43]:
from llama_index.chat_engine import SimpleChatEngine
chat_engine = SimpleChatEngine.from_defaults()
chat_engine.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Assistant: Hello there! I'm just a friendly AI here to help answer any questions you might have. How can I assist you today? If you have any specific query, feel free to ask and I'll do my best to provide you with accurate and helpful information.

As for the greeting, "yo wassup bro," it's a casual way of saying "hello, how are you?" in English, commonly used among friends or in informal settings. It's originated from hip hop culture. Let me know if you have any other question or if there's something else I can help you with! :)



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Assistant: I'd be happy to help you understand the content of a document, but I'll need you to provide the document itself or at least some context about it first. The title, author, and any relevant information you have about the document will be helpful in determining its content.

For example, if the document is a business proposal, I might look for information about the company, the product or service being proposed, and the target audience. If it's a research paper, I might look for the topic, the author's thesis statement, and the research methods used.

If you can provide me with some details about the document, I'll do my best to give you an accurate summary or explanation. If you don't have the document itself, you can try describing its purpose, the format, and any key details you remember. Let me know if you have any questions or need further clarification! :)



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
print(response)

In [ ]:
print(chat_engine.get_prompts()['response_synthesizer:text_qa_template'])

In [ ]:
print(response. )

In [ ]:
streaming_response = chat_engine.stream_chat("what is the name of the movie? Who is the director of the movie?")
for token in streaming_response.response_gen:
    print(token, end="")

In [ ]:

import gradio as gr

gr.ChatInterface(predict).launch(share=True)